# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846826102106                   -0.70    4.8         
  2   -7.852319464371       -2.26       -1.53    1.0   24.7ms
  3   -7.852613714145       -3.53       -2.56    1.5   27.3ms
  4   -7.852645955095       -4.49       -2.88    2.8   36.1ms
  5   -7.852646484056       -6.28       -3.15    1.0   26.0ms
  6   -7.852646679302       -6.71       -4.05    1.0   25.2ms
  7   -7.852646686601       -8.14       -5.20    1.8   30.4ms
  8   -7.852646686723       -9.91       -5.41    2.0   30.2ms
  9   -7.852646686727      -11.40       -5.59    1.2   26.2ms
 10   -7.852646686730      -11.60       -6.74    1.0   25.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846879339739                   -0.70           4.5         
  2   -7.852529025027       -2.25       -1.64   0.80    2.5    269ms
  3   -7.852635433059       -3.97       -2.75   0.80    1.0   23.6ms
  4   -7.852646421261       -4.96       -3.22   0.80    1.8   27.8ms
  5   -7.852646661730       -6.62       -4.00   0.80    1.0   22.6ms
  6   -7.852646686384       -7.61       -4.56   0.80    2.0   28.9ms
  7   -7.852646686704       -9.49       -5.52   0.80    1.2   25.3ms
  8   -7.852646686729      -10.60       -6.39   0.80    2.2   30.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.393751e+01     3.240481e+00
 * time: 0.05931401252746582
     1     1.305702e+00     1.821923e+00
 * time: 0.2817058563232422
     2    -1.729064e+00     2.219201e+00
 * time: 0.30689096450805664
     3    -3.591691e+00     1.927514e+00
 * time: 0.3426978588104248
     4    -4.966495e+00     1.947827e+00
 * time: 0.37777090072631836
     5    -6.654111e+00     9.992025e-01
 * time: 0.41257691383361816
     6    -7.387702e+00     4.386107e-01
 * time: 0.44777393341064453
     7    -7.644966e+00     2.188102e-01
 * time: 0.4721829891204834
     8    -7.746144e+00     1.085700e-01
 * time: 0.4970219135284424
     9    -7.801247e+00     1.487940e-01
 * time: 0.5216610431671143
    10    -7.822715e+00     8.661438e-02
 * time: 0.546440839767456
    11    -7.838794e+00     6.960334e-02
 * time: 0.5714459419250488
    12    -7.845025e+00     5.442257e-02
 * time: 0.5961189270019531
    13    -7.848315e+00     3.705262e-02
 * time: 0.621520

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846869975747                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645910457                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.92s
  3   -7.852646686730      -13.30       -7.24    197ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.2759983990436503e-7
|ρ_newton - ρ_scfv| = 4.950090790746194e-7
|ρ_newton - ρ_dm|   = 1.3144871858226076e-9
